In [4]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import csv
import warnings
import glob

In [5]:
warnings.filterwarnings("ignore")

In [6]:
def plot_columns(df):
    colors = plt.cm.tab20(range(df.shape[1]))  

    for i, column in enumerate(df.columns):
        color = colors[i % 24]

        plt.figure()
        plt.plot(df.index, df[column], color=color)
        plt.xlabel('Ano')
        plt.ylabel('Valor')
        plt.title(column)

    plt.show()

In [7]:
def centered_moving_average(column, window_action, window_size):
    start_index, end_index = window_action
    data = column.to_numpy()

    data_length = len(data)

    for i in range(start_index, end_index + 1):
        if not np.isnan(data[i]):
            init = data[i]
            start = max(0, i - window_size)
            end = min(data_length, i + window_size + 1)
            window = data[start:end]
            valid_values = [value for value in window if not pd.isna(value)]
            if len(valid_values) > 0:
                data[i] = np.mean(valid_values)

    modified_column = pd.Series(data, name=column.name)

    return modified_column

In [8]:
# Linear Interpolation 
def ends_filler(column):
    if  column.dtype.kind in ['i', 'f']:
            column = first_filler(column)
            column = last_filler(column)
    return column

def first_filler(column):
    if pd.isnull(column.at[0]) or np.isnan(column.at[0]):
        size = len(column) -1
        first_valid_index = 0
        while first_valid_index <= size and (pd.isnull(column.at[first_valid_index]) or np.isnan(column.at[first_valid_index])):
            first_valid_index += 1
        if first_valid_index > size:
            column.at[0] = 0.0
        else:
            column.at[0] = column.at[first_valid_index]
    return column
    
def last_filler(column):
    if pd.isnull(column.at[len(column) -1]) or np.isnan(column.at[len(column) -1]):  
        last_valid_index = len(column) -1
        while last_valid_index >= 0 and (pd.isnull(column.at[last_valid_index]) or np.isnan(column.at[last_valid_index])):
            last_valid_index -= 1
        if last_valid_index < 0:
            column.at[len(column) -1] = 0.0
        else:
            column.at[len(column) -1] = column.at[last_valid_index]
    return column
    
def prev_valid_index(column, pos):
    prev_index = pos
    size = len(column) -1
    if pos > 0 and pos < size:
        while prev_index >= 0 and (pd.isnull(column.at[prev_index]) or np.isnan(column.at[prev_index])):
            prev_index -= 1
    return prev_index  

def next_valid_index(column, pos):
    next_index = pos
    size = len(column) -1
    if pos > 0 and pos < size:
        while next_index <= size and (pd.isnull(column.at[next_index]) or np.isnan(column.at[next_index])):
            next_index += 1
    return next_index

def linear_filler(column):
    if column.isnull().any():
        for i, value in enumerate(column):
            if i > 0 and (pd.isnull(value) or np.isnan(value) or (value == '')):
                start = prev_valid_index(column, i)
                end = next_valid_index(column, i)
                gap = list(range(start + 1, end))
                linear_a = (column.at[end] - column.at[start])/(end - start)
                linear_b = column.at[start] - linear_a*start
                for x in gap:
                    column.at[x] = linear_a*x + linear_b
    return column 

In [9]:
def read_csv_replace_missing_by_linear(file_path, skiped_rows):
    with open(file_path, 'r') as f:
        
        df = pd.read_csv(file_path, skiprows=skiped_rows)
        
        i = 0
        for col in df.columns: # Elimina missing values por médias e transforma em float
            if i > 0:
                df[col] = df[col].replace('-9999', np.nan)
                df[col] = df[col].replace('-', np.nan)
                df[col] = df[col].replace('', np.nan).astype(float)
                df[col] = ends_filler(df[col])
                df[col] = linear_filler(df[col])
#                 df[col] = centered_moving_average(df[col], (0, len(df[col]) - 1), 24) #2 anos
            i += 1
        return df

In [10]:
df = pd.read_csv("consumo_cimento_mensal.csv")
df

,Ano,Rondônia - Consumo de Cimento (t),Acre - Consumo de Cimento (t),Amazonas - Consumo de Cimento (t),Roraima - Consumo de Cimento (t),Pará - Consumo de Cimento (t),Amapá - Consumo de Cimento (t),Totins-01 - Consumo de Cimento (t),Mahão -01- Consumo de Cimento (t),Piauí - Consumo de Cimento (t),...,Espírito to --01 Consumo de Cimento (t),Rio De eiro-01 - Consumo de Cimento (t),São Paulo - Consumo de Cimento (t),Paá - -01Consumo de Cimento (t),ta C-01atarina - Consumo de Cimento (t),Rio Gde D-01o Sul - Consumo de Cimento (t),Mato Grosso - Consumo de Cimento (t),Mato Grosso Do Sul - Consumo de Cimento (t),Goiás - Consumo de Cimento (t),Distrito Federal - Consumo de Cimento (t)
0,2003-01,13.867,3.260,28.193,6.920,84.701,10.392,17.555,43.342,21.844,...,48.426,254.362,782.111,189.804,142.456,188.093,47.470,33.744,97.447,50.047
1,2003-02,10.777,2.191,31.226,7.476,80.221,6.857,14.976,28.663,15.814,...,54.969,252.842,789.891,174.713,135.829,170.973,45.387,32.230,97.256,48.110
2,2003-03,13.888,2.154,33.484,8.659,76.122,7.011,13.334,27.900,16.067,...,53.847,229.385,774.309,180.801,133.135,174.880,44.907,33.559,101.442,49.006
3,2003-04,11.859,2.643,31.596,7.259,76.544,7.122,15.173,25.739,14.938,...,50.375,243.407,782.071,180.172,144.723,178.111,45.467,34.283,98.217,48.445
4,2003-05,14.431,2.546,32.919,6.786,83.445,5.267,18.728,30.203,17.765,...,52.576,256.535,841.400,185.597,150.523,184.331,56.246,36.066,112.540,51.436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-08,49.066,21.748,66.077,9.133,156.442,13.208,54.154,102.698,77.776,...,100.170,241.702,1112.914,412.177,377.545,288.217,192.533,103.854,274.673,77.448
236,2022-09,49.850,20.032,58.590,10.622,145.777,13.476,55.664,98.133,72.377,...,85.832,221.036,1028.144,337.056,370.282,272.178,183.895,95.890,247.240,64.321
237,2022-10,43.733,18.573,58.793,10.300,139.233,11.236,52.320,94.964,71.791,...,91.318,224.399,996.449,340.018,340.900,272.665,180.525,89.679,246.647,72.839
238,2022-11,40.078,15.493,53.509,10.258,142.025,13.549,43.036,97.684,71.015,...,77.187,221.319,1024.272,383.477,363.900,275.972,161.683,99.356,225.813,59.598


In [11]:
df = read_csv_replace_missing_by_linear("consumo_cimento_mensal.csv",0)
df

,Ano,Rondônia - Consumo de Cimento (t),Acre - Consumo de Cimento (t),Amazonas - Consumo de Cimento (t),Roraima - Consumo de Cimento (t),Pará - Consumo de Cimento (t),Amapá - Consumo de Cimento (t),Totins-01 - Consumo de Cimento (t),Mahão -01- Consumo de Cimento (t),Piauí - Consumo de Cimento (t),...,Espírito to --01 Consumo de Cimento (t),Rio De eiro-01 - Consumo de Cimento (t),São Paulo - Consumo de Cimento (t),Paá - -01Consumo de Cimento (t),ta C-01atarina - Consumo de Cimento (t),Rio Gde D-01o Sul - Consumo de Cimento (t),Mato Grosso - Consumo de Cimento (t),Mato Grosso Do Sul - Consumo de Cimento (t),Goiás - Consumo de Cimento (t),Distrito Federal - Consumo de Cimento (t)
0,2003-01,13.867,3.260,28.193,6.920,84.701,10.392,17.555,43.342,21.844,...,48.426,254.362,782.111,189.804,142.456,188.093,47.470,33.744,97.447,50.047
1,2003-02,10.777,2.191,31.226,7.476,80.221,6.857,14.976,28.663,15.814,...,54.969,252.842,789.891,174.713,135.829,170.973,45.387,32.230,97.256,48.110
2,2003-03,13.888,2.154,33.484,8.659,76.122,7.011,13.334,27.900,16.067,...,53.847,229.385,774.309,180.801,133.135,174.880,44.907,33.559,101.442,49.006
3,2003-04,11.859,2.643,31.596,7.259,76.544,7.122,15.173,25.739,14.938,...,50.375,243.407,782.071,180.172,144.723,178.111,45.467,34.283,98.217,48.445
4,2003-05,14.431,2.546,32.919,6.786,83.445,5.267,18.728,30.203,17.765,...,52.576,256.535,841.400,185.597,150.523,184.331,56.246,36.066,112.540,51.436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-08,49.066,21.748,66.077,9.133,156.442,13.208,54.154,102.698,77.776,...,100.170,241.702,1112.914,412.177,377.545,288.217,192.533,103.854,274.673,77.448
236,2022-09,49.850,20.032,58.590,10.622,145.777,13.476,55.664,98.133,72.377,...,85.832,221.036,1028.144,337.056,370.282,272.178,183.895,95.890,247.240,64.321
237,2022-10,43.733,18.573,58.793,10.300,139.233,11.236,52.320,94.964,71.791,...,91.318,224.399,996.449,340.018,340.900,272.665,180.525,89.679,246.647,72.839
238,2022-11,40.078,15.493,53.509,10.258,142.025,13.549,43.036,97.684,71.015,...,77.187,221.319,1024.272,383.477,363.900,275.972,161.683,99.356,225.813,59.598


In [12]:
df_year = df.copy()
df_year['Ano'] = df_year['Ano'].str[:4]
df_year = df_year.groupby('Ano').sum()
df_year

,Rondônia - Consumo de Cimento (t),Acre - Consumo de Cimento (t),Amazonas - Consumo de Cimento (t),Roraima - Consumo de Cimento (t),Pará - Consumo de Cimento (t),Amapá - Consumo de Cimento (t),Totins-01 - Consumo de Cimento (t),Mahão -01- Consumo de Cimento (t),Piauí - Consumo de Cimento (t),Ceará - Consumo de Cimento (t),...,Espírito to --01 Consumo de Cimento (t),Rio De eiro-01 - Consumo de Cimento (t),São Paulo - Consumo de Cimento (t),Paá - -01Consumo de Cimento (t),ta C-01atarina - Consumo de Cimento (t),Rio Gde D-01o Sul - Consumo de Cimento (t),Mato Grosso - Consumo de Cimento (t),Mato Grosso Do Sul - Consumo de Cimento (t),Goiás - Consumo de Cimento (t),Distrito Federal - Consumo de Cimento (t)
Ano,,,,,,,,,,,,,,,,,,,,,
2003,185.386000,43.603000,418.347000,74.283000,1052.310000,85.0930,239.493000,470.726000,238.886000,888.572000,...,635.153000,3021.269,9528.051000,2158.499000,1687.172000,2103.000000,668.575000,417.592000,1311.935000,635.841000
2004,209.004000,58.142000,587.382000,39.775000,1193.163000,80.0080,397.426000,531.005000,276.711000,917.163000,...,636.659000,3092.530,9148.318000,2259.910000,1740.105000,2116.716000,716.682000,466.165000,1514.939000,771.216000
2005,233.057000,80.914000,613.286000,30.179000,1264.982000,104.1970,431.203000,579.458000,272.764000,986.162000,...,768.967000,3324.329,9724.722000,2225.422000,1824.650000,1966.059000,737.634000,479.481000,1716.900000,855.531000
2006,272.001000,114.415000,626.147000,34.243000,1181.617000,115.5400,312.507000,693.939000,333.178000,1055.446000,...,673.561000,3665.682,10863.790000,2203.646000,1870.478000,2106.926000,701.548000,515.485000,1730.142000,747.184000
2007,351.377000,121.872000,666.931000,35.931000,1272.803000,135.0460,426.737000,816.847000,386.585000,1173.764000,...,974.754000,4107.097,11775.453000,2382.720000,2081.839000,2280.943000,840.901000,632.929000,1943.769000,808.342000
2008,390.788000,119.145000,769.233000,60.009000,1559.491000,116.3660,450.745000,963.205000,449.076000,1328.591000,...,1263.713000,4300.676,13476.268000,3001.521000,2653.587000,2718.542000,994.957000,814.771000,2310.616000,909.713000
2009,452.325000,121.146000,744.252000,63.064000,1355.544000,112.7290,467.588000,1123.586000,536.766000,1437.485000,...,1138.824000,3966.930,13304.301000,3092.068000,2669.400000,2685.740000,998.465000,693.327000,2335.717000,991.263000
2010,921.425000,174.705000,796.116000,86.408000,1640.682000,151.5420,487.208000,1378.741000,639.535000,1794.240000,...,1223.466000,4199.911,14534.980000,3749.448000,2852.759000,3070.312000,1099.340000,813.149000,2657.527000,1167.604000
2011,1021.570000,178.322000,948.011000,103.009000,1856.176000,161.9700,458.331000,1282.016000,625.022000,1717.740000,...,1271.961000,4390.254,14871.943000,4174.942000,3087.810000,3399.177000,1204.074000,923.248000,2901.707000,1277.890000


In [13]:
df_year.index = df_year.index.astype(int)
df_2014 = df_year[df_year.index >= 2014] 
df_2014

,Rondônia - Consumo de Cimento (t),Acre - Consumo de Cimento (t),Amazonas - Consumo de Cimento (t),Roraima - Consumo de Cimento (t),Pará - Consumo de Cimento (t),Amapá - Consumo de Cimento (t),Totins-01 - Consumo de Cimento (t),Mahão -01- Consumo de Cimento (t),Piauí - Consumo de Cimento (t),Ceará - Consumo de Cimento (t),...,Espírito to --01 Consumo de Cimento (t),Rio De eiro-01 - Consumo de Cimento (t),São Paulo - Consumo de Cimento (t),Paá - -01Consumo de Cimento (t),ta C-01atarina - Consumo de Cimento (t),Rio Gde D-01o Sul - Consumo de Cimento (t),Mato Grosso - Consumo de Cimento (t),Mato Grosso Do Sul - Consumo de Cimento (t),Goiás - Consumo de Cimento (t),Distrito Federal - Consumo de Cimento (t)
Ano,,,,,,,,,,,,,,,,,,,,,
2014,614.394714,171.709286,771.567857,120.417571,2041.730714,227.2430,426.767286,1303.411571,755.713857,1964.413143,...,1296.041857,4454.468,13714.511286,4341.821857,3246.749286,3226.786714,1411.234429,995.222143,2861.897429,979.998857
2015,510.273771,151.486629,746.867486,113.282057,1863.549771,198.0696,454.929429,1191.509657,794.945486,2037.703714,...,1250.323886,3639.072,11767.653429,4108.786286,3017.084229,2978.617371,1241.100343,951.041314,2517.290743,811.194686
2016,410.580514,134.223086,739.457657,105.481371,1705.680514,166.8504,489.016286,1100.217771,842.687657,2146.547143,...,1210.917257,2811.648,9811.084286,3869.334857,2776.217486,2744.168914,1083.128229,904.537543,2193.545829,638.818457
2017,406.814000,150.491000,687.211000,99.723000,1564.940000,135.0210,500.270000,971.921000,831.356000,2035.158000,...,1097.369000,2284.918,9918.501000,3764.518000,2891.489000,2703.746000,1220.058000,870.341000,2270.346000,584.646000
2018,376.138000,148.696000,350.177000,83.012000,1375.281000,123.3470,396.438000,750.012000,626.223000,1758.900000,...,1146.390000,2391.212,10001.077000,3773.354000,2976.485000,2665.575000,1316.386000,869.680000,2282.903000,571.130000
2019,399.441000,153.806000,533.444000,121.120000,1290.928000,120.1530,402.932000,774.045000,702.437000,1701.230000,...,1139.906000,2715.843,10458.276000,3786.974000,3258.027000,2753.564000,1446.064000,902.144000,2367.716000,637.028000
2020,488.194000,178.033000,595.320000,151.571000,1444.547000,133.5230,500.126000,1038.249000,788.164000,1864.364000,...,1138.655000,2935.607,11181.809000,4235.532000,3537.032000,2993.250000,1555.266000,1038.045000,2438.585000,662.625000
2021,502.824000,189.357000,610.155000,144.989000,1529.052000,157.6210,542.030000,1059.129000,826.368000,2127.104000,...,1135.022000,2829.682,12113.838000,4784.413000,4089.333000,3222.295000,1788.146000,1092.056000,2782.135000,789.947000
2022,501.142000,202.321000,640.274000,132.217000,1592.624000,142.7390,554.321000,1083.999000,804.417000,1966.646000,...,1040.260000,2682.995,12383.649000,4538.256000,4255.808000,3135.029000,1947.017000,1152.060000,2807.973000,786.865000


In [14]:
for column in df_2014.columns:
    file_name = f"2014_01_cons_cim_{column}.csv"
    column_df = df_2014[[column]]
    column_df.to_csv(file_name, index=True)